# APIM ❤️ Foundry

## Microsoft Foundry Private connectivity lab
![flow](./architecture.png)

Playground to show how to create a private network for consuming REST API managed as MCPs using `Foundry Agents V1`. This lab demonstrates how to create a private network for consuming MCPs from `Foundry Agents V1` using `Private Link Services`, `Azure API Management (APIM)`, and `Azure Front Door`.

Notes:
- `Foundry Agents` are only accessible through `Private Endpoints`. Public network access is disabled.
- `Azure API Management` exposing REST API managed as a remote Model Context Protocol (MCP). [https://learn.microsoft.com/en-us/azure/api-management/export-rest-mcp-server]
- `Azure API Management` is not accessible from a public network. The only access is through `Azure Front Door`.
- `Azure Front Door` manages the traffic to the `Azure API Management` service through `Private Link Service`.
- **`Azure Front Door` is the only publicly-accessible service.**

### Prerequisites
- [Python 3.12 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [3]:
import os, sys, json
sys.path.insert(1, '../../shared')
import utils
import random
import string

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"

apim_sku = 'Standardv2'
apic_location = 'eastus'

# Prioritize East US until exhaustion (simulate PTU with TPM), then equally distribute between Sweden and West US (consumption fallback)
models_config = [
    {"name": "gpt-4.1", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 20}
]

app_registration_name = f"lab-{deployment_name}-{''.join(random.choices(string.ascii_letters + string.digits, k=8)).lower()}-app"

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 12:54:09.769886 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations.

In [6]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "modelsConfig": { "value": models_config },
        "apimPublicNetworkAccess": { "value": "Enabled" },
        "apicLocation": { "value": apic_location },
        "mcpEntraAppName": { "value": app_registration_name }
    }
}

# Write the parameters to the params.json file
with open('./params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file ./main.bicep --parameters ./params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-ai-foundry-private-mcp 
👉🏽 Using existing resource group 'lab-ai-foundry-private-mcp'
⚙️ Running: az deployment group create --name ai-foundry-private-mcp --resource-group lab-ai-foundry-private-mcp --template-file ./main.bicep --parameters ./params.json 
✅ Deployment 'ai-foundry-private-mcp' succeeded ⌚ 13:08:15.208731 :45s]


<a id='3'></a>
### 3️⃣ Approve Front Door private link connection to APIM

In the deployed Bicep template, Azure Front Door will establish a private link connection to the API Management service. This connection should be approved. Run the following command to approve the connection.

In [28]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_resource_id = utils.get_deployment_output(output, 'apimResourceId', 'apimResourceId')

    outputPls = utils.run(f"az network private-endpoint-connection list --id {apim_resource_id} --query [?properties.privateLinkServiceConnectionState.status=='Pending'].id --output tsv")

    if outputPls.success:
        pls_connection_id = outputPls.text
        print(pls_connection_id)
        utils.run(f"az network private-endpoint-connection approve --id {pls_connection_id} --description 'Approved'", f"Private Link Connection approved.", f"Failed to approve Private Link Connection: {pls_connection_id}")

⚙️ Running: az deployment group show --name ai-foundry-private-mcp -g lab-ai-foundry-private-mcp 
✅ Retrieved deployment: ai-foundry-private-mcp ⌚ 14:12:29.092920 :4s]
👉🏽 apimResourceId: /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-ai-foundry-private-mcp/providers/Microsoft.ApiManagement/service/apim-blbmsxvxry2wc
⚙️ Running: az network private-endpoint-connection list --id /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-ai-foundry-private-mcp/providers/Microsoft.ApiManagement/service/apim-blbmsxvxry2wc --query [?properties.privateLinkServiceConnectionState.status=='Pending'].id --output tsv 

⚙️ Running: az network private-endpoint-connection approve --id  --description 'Approved' 
❌ Failed to approve Private Link Connection:  ⌚ 14:12:41.744682 :6s] ERROR: argument --id: expected one argument

Examples from AI knowledge base:
az network private-endpoint-connection approve --id /subscriptions/00000000-0000-0000-0000-000000000000/resource

<a id='4'></a>
### 4️⃣ Disabling APIM public network access

As of May 2025, during the creation, the `APIM` service cannot disable the public network access. This behavior might change in the future. As a workaround, you can disable the public network access after the deployment is completed. To do that, you can run the Bicep template again after changing the `apimPublicNetworkAccess` parameter to `Disabled`. This will update the APIM service and disable the public network access.

In [31]:
bicep_parameters['parameters']['apimPublicNetworkAccess']['value'] = "Disabled"

# Write the parameters to the params.json file
with open('./params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file ./main.bicep --parameters ./params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az deployment group create --name ai-foundry-private-mcp --resource-group lab-ai-foundry-private-mcp --template-file ./main.bicep --parameters ./params.json 
✅ Deployment 'ai-foundry-private-mcp' succeeded ⌚ 14:19:06.452141 :55s]


<a id='5'></a>
### 5️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [14]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    frontdoor_endpoint = utils.get_deployment_output(output, 'frontDoorEndpointHostName', 'Front Door Endpoint')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)
    ai_foundry_project_endpoint = utils.get_deployment_output(output, 'ai_project_endpoint', 'AI Foundry Project Endpoint')
    ai_deployment_name = utils.get_deployment_output(output, 'ai_model_deployment_name', 'AI Model Deployment Name')

⚙️ Running: az deployment group show --name ai-foundry-private-mcp -g lab-ai-foundry-private-mcp 
✅ Retrieved deployment: ai-foundry-private-mcp ⌚ 13:44:00.372570 :4s]
👉🏽 Front Door Endpoint: afd-qbnxbqr3rdwaq-cwcchbc2gqamhma4.b01.azurefd.net
👉🏽 APIM API Gateway URL: https://apim-blbmsxvxry2wc.azure-api.net
👉🏽 APIM Subscription Key (masked): ****6fa6
👉🏽 AI Foundry Project Endpoint: https://foundryblbmsxvxry2wc.services.ai.azure.com/api/projects/foundry-project-blbmsxvxry2wc
👉🏽 AI Model Deployment Name: gpt-4.1


<a id='6'></a>
### 6️⃣ 🧪 Test the API using a direct HTTP call through Frontdoor

Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

In [ ]:
import requests

def call_mcp_api(resource_url):
    url = f"{resource_url}/order-mcp/mcp"
    request = {"method":"tools/call","params":{"name":"PlaceOrder-invoke","arguments":{"request-PlaceOrder":{"sku":"sku-123","quantity":5}}},"jsonrpc":"2.0","id":1}
    print(f"Calling MCP endpoint at URL: {url}")
    utils.print_info("Calling MCP endpoint WITH authorization...")
    output = utils.run("az account get-access-token --resource \"https://azure-api.net/authorization-manager\"")

    access_token = output.json_data['accessToken']
    response = requests.post(url, stream=True, headers={"Content-Type": "application/json", "agent-id": "Agent1", "Authorization": "Bearer " + str(access_token)}, json=request)

    print(f"Response status code: {response.status_code}")

    if response.status_code == 200:
        utils.print_ok("Received status code 200 as expected")

        print("Response:")
        for line in response.iter_lines(decode_unicode=True):
            if line:
                if (line == 'event: close'):
                    response.close()
                    break
                if (line.startswith('data')):
                    data = json.loads(line.strip()[5:])
                    print(json.dumps(json.loads(data["result"]["content"][0]["text"]), indent=4))
    else:
        utils.print_error(f"Unexpected status code: {response.status_code}. Response text: {response.text}")

call_mcp_api(f"https://{frontdoor_endpoint}")

Calling MCP endpoint at URL: https://afd-qbnxbqr3rdwaq-cwcchbc2gqamhma4.b01.azurefd.net/order-mcp/mcp
👉🏽 Calling MCP endpoint WITH authorization...
⚙️ Running: az account get-access-token --resource "https://azure-api.net/authorization-manager" 
https://afd-qbnxbqr3rdwaq-cwcchbc2gqamhma4.b01.azurefd.net/order-mcp/mcp
Response status code: 200
✅ Received status code 200 as expected ⌚ 14:22:31.701364 
Response:
{
    "status": "Order placed with id 4231 for SKU sku-123 with 5 items."
}


<a id='7'></a>
### 7️⃣ 🧪 Test APIM API access through public network

APIM is not accessible from public network. The only way to access it is through Azure Front Door. This test should fail with a 403 error.

In [33]:
call_mcp_api(apim_resource_gateway_url)

Calling MCP endpoint at URL: https://apim-blbmsxvxry2wc.azure-api.net/order-mcp/mcp
👉🏽 Calling MCP endpoint WITH authorization...
⚙️ Running: az account get-access-token --resource "https://azure-api.net/authorization-manager" 
https://apim-blbmsxvxry2wc.azure-api.net/order-mcp/mcp
Response status code: 403
❌ Unexpected status code: 403. Response text: { "statusCode": 403, "message": "Request originated from client public IP address 177.133.186.193, public network access on this `Microsoft.ApiManagement/service/apim-blbmsxvxry2wc` is disabled. To connect to `Microsoft.ApiManagement/service/apim-blbmsxvxry2wc`, please use the Private Endpoint from inside your virtual network. To learn more https://aka.ms/apim-privateendpoint " } ⌚ 14:22:49.721215 


<a id='8'></a>
### 8️⃣ 🧪 Test MCP Client Using the Foundry SDK

The Microsoft Foundry resource is accessible in the private virtual network through a Private Endpoint. This means that any resource within this network can communicate securely with any of the Foundry resources, including an agent created using the Foundry Agent service. 

To validate this behavior, the Bicep template creates an Azure virtual machine that will act as a jumpbox. This VM is deployed in the same virtual network as the Foundry resource and has access to it. 

The following instructions will help you run a Foundry Agent inside the VM, and verify that it can connect to the MCP server:



##### 1. Connect to Jumpbox
Use Azure Bastion to connect to the VM:

```bash
az network bastion ssh \
  --name bastion-host \
  --resource-group <resource-group> \
  --target-resource-id <vm-resource-id> \
  --auth-type password \
  --username azureuser
```

Or connect via the Azure Portal: Navigate to the VM → Connect → Bastion

Once connected to the jumpbox, create the required scripts:

##### 2. Create `scripts/load_env_from_kv.py`
```bash 
cat > ~/scripts/load_env_from_kv.py << 'EOF'
# Paste content of agent/load_env_from_kv.py here
EOF
```


##### 3. Create `scripts/sample_agents_mcp.py`
```bash 
cat > ~/scripts/sample_agents_mcp.py << 'EOF'
# Paste content of agent/sample_agents_mcp.py here
EOF
```

##### 4. Run the agent
```bash
# Activate virtual environment
source ~/venv/bin/activate

# Get Key Vault URL from deployment outputs
KEY_VAULT_URL="https://kv-xxxxx.vault.azure.net/"

# Run the MCP agent
python3 ~/scripts/sample_agents_mcp.py $KEY_VAULT_URL
```



<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.